In [61]:
import pandas as pd

In [62]:
data = pd.read_csv('HR_comma_sep.csv.txt')

In [63]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [64]:
data.rename(columns={'sales':'dept'}, inplace=True)

In [65]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,dept,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [66]:
X=data.drop(['left'],1)

C:\Users\vip\AppData\Local\Temp\ipykernel_10740\3797126929.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X=data.drop(['left'],1)


In [67]:
Y=data['left']

In [68]:
print(X.dtypes)

satisfaction_level       float64
last_evaluation          float64
number_project             int64
average_montly_hours       int64
time_spend_company         int64
Work_accident              int64
promotion_last_5years      int64
dept                      object
salary                    object
dtype: object


In [69]:
float_data=X.select_dtypes(include=['float'])

In [70]:
int_data=X.select_dtypes(include=['int64'])

In [71]:
obj_data=X.select_dtypes(include=['object'])

In [72]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder,MinMaxScaler,OneHotEncoder
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestClassifier

In [73]:
int_pipeline=make_pipeline(MinMaxScaler(),SelectKBest(k=4,score_func=f_classif))
obj1_pipeline=make_pipeline(OneHotEncoder())
obj2_pipeline=make_pipeline(OrdinalEncoder())

preprocessor=make_column_transformer(
    (int_pipeline,int_data.columns),
    (obj1_pipeline,['dept']),
    (obj2_pipeline,['salary']),
    remainder="passthrough"
)


In [77]:
master_pipeline=make_pipeline(preprocessor,RandomForestClassifier(n_estimators=10))

In [78]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=0)

In [79]:
master_pipeline.fit(X_train,Y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('minmaxscaler',
                                                                   MinMaxScaler()),
                                                                  ('selectkbest',
                                                                   SelectKBest(k=4))]),
                                                  Index(['number_project', 'average_montly_hours', 'time_spend_company',
       'Work_accident', 'promotion_last_5years'],
      dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['dept']),
                                                 ('pipeline-3',
                                                  Pipeline(steps=[('ordinalencoder',
                                                                   OrdinalEncoder())]),
                                                  ['salary'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators=10))])

In [80]:
y_pred=master_pipeline.predict(X_test)

In [81]:
master_pipeline.score(X_test,Y_test)

0.9862222222222222

In [82]:
print(master_pipeline.steps[0][1].transformers)

[('pipeline-1', Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('selectkbest', SelectKBest(k=4))]), Index(['number_project', 'average_montly_hours', 'time_spend_company',
       'Work_accident', 'promotion_last_5years'],
      dtype='object')), ('pipeline-2', Pipeline(steps=[('onehotencoder', OneHotEncoder())]), ['dept']), ('pipeline-3', Pipeline(steps=[('ordinalencoder', OrdinalEncoder())]), ['salary'])]


In [60]:
print(master_pipeline)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('minmaxscaler',
                                                                   MinMaxScaler()),
                                                                  ('selectkbest',
                                                                   SelectKBest(k=4))]),
                                                  Index(['number_project', 'average_montly_hours', 'time_spend_company',
       'Work_accident', 'promotion_last_5years'],
      dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['dept']),
          

In [83]:
from sklearn.model_selection import GridSearchCV

In [84]:
params={'columntransformer__pipeline-1__selectkbest__k':[2,3,4],
        'randomforestclassifier__n_estimators':[100,120,140],
       }
gs=GridSearchCV(master_pipeline,param_grid=params,cv=4)
gs.fit(X_train,Y_train)


GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('minmaxscaler',
                                                                                          MinMaxScaler()),
                                                                                         ('selectkbest',
                                                                                          SelectKBest(k=4))]),
                                                                         Index(['number_project', 'average_montly_hours', 'time_spend_company',
       'Work_accident', 'promotion_last_5years'],
      dtype='object')),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('onehotencoder',
                                                                                          OneHotEncoder())]),
                                                                         ['dept']),
                                                                        ('pipeline-3',
                                                                         Pipeline(steps=[('ordinalencoder',
                                                                                          OrdinalEncoder())]),
                                                                         ['salary'])])),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(n_estimators=10))]),
             param_grid={'columntransformer__pipeline-1__selectkbest__k': [2, 3,
                                                                           4],
                         'randomforestclassifier__n_estimators': [100, 120,
                                                                  140]})

In [85]:
print(gs.best_params_)

{'columntransformer__pipeline-1__selectkbest__k': 3, 'randomforestclassifier__n_estimators': 120}


In [86]:
print(gs.score(X_test,Y_test))

0.9875555555555555
